## Step 1: Prepare Event Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def generate_synthetic_events(num_events, sensor_width, sensor_height):
    timestamps = np.random.uniform(0, 1, num_events)
    x_coords = np.random.randint(0, sensor_width, num_events)
    y_coords = np.random.randint(0, sensor_height, num_events)
    polarities = np.random.choice([-1, 1], num_events)
    events = np.vstack((timestamps, x_coords, y_coords, polarities)).T
    return events

def normalize_events(events, sensor_width, sensor_height):
    min_t, max_t = events[:, 0].min(), events[:, 0].max()
    events[:, 0] = (events[:, 0] - min_t) / (max_t - min_t)
    events[:, 1] = events[:, 1] / sensor_width
    events[:, 2] = events[:, 2] / sensor_height
    return events

# Generate synthetic events
num_events = 1000
sensor_width = 240
sensor_height = 180
events = generate_synthetic_events(num_events, sensor_width, sensor_height)

# Normalize events
normalized_events = normalize_events(events, sensor_width, sensor_height)


## Step 2: Modify the Network to Handle Event Data

In [2]:
class LIFNeuronLayer:
    def __init__(self, num_neurons, tau_m, V_rest, V_thresh, V_reset, R, dt):
        self.num_neurons = num_neurons
        self.tau_m = tau_m
        self.V_rest = V_rest
        self.V_thresh = V_thresh
        self.V_reset = V_reset
        self.R = R
        self.dt = dt
        self.V = np.full(num_neurons, V_rest)
        self.spikes = np.zeros(num_neurons)

    def update(self, I):
        dV = (-(self.V - self.V_rest) + self.R * I) * (self.dt / self.tau_m)
        self.V += dV

        self.spikes = self.V >= self.V_thresh
        self.V[self.spikes] = self.V_reset
    
    def reset(self):
        self.V = np.full(self.num_neurons, self.V_rest)
        self.spikes = np.zeros(self.num_neurons)

class SpikingNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, tau_m, V_rest, V_thresh, V_reset, R, dt):
        self.input_layer = np.zeros(input_size)
        self.hidden_layer = LIFNeuronLayer(hidden_size, tau_m, V_rest, V_thresh, V_reset, R, dt)
        self.output_layer = LIFNeuronLayer(output_size, tau_m, V_rest, V_thresh, V_reset, R, dt)
        
        self.W_input_hidden = np.random.normal(0, 1, (hidden_size, input_size))
        self.W_hidden_output = np.random.normal(0, 1, (output_size, hidden_size))

    def forward(self, input_spikes):
        self.input_layer = input_spikes

        I_hidden = self.W_input_hidden @ self.input_layer
        self.hidden_layer.update(I_hidden)

        I_output = self.W_hidden_output @ self.hidden_layer.spikes
        self.output_layer.update(I_output)

    def reset(self):
        self.hidden_layer.reset()
        self.output_layer.reset()

def events_to_input(events, num_neurons, T, dt):
    time_steps = int(T / dt)
    input_spikes = np.zeros((num_neurons, time_steps))

    for event in events:
        t_event = event[0] * T  # Scale event time to simulation duration
        neuron_idx = int(event[1] * num_neurons)
        time_idx = int(t_event / dt)

        # Ensure indices are within bounds
        if neuron_idx >= num_neurons:
            neuron_idx = num_neurons - 1
        if time_idx >= time_steps:
            time_idx = time_steps - 1

        input_spikes[neuron_idx, time_idx] += event[3]  # Polarity as spike contribution

    return input_spikes


# Parameters
tau_m = 20e-3  # Membrane time constant
V_rest = -70e-3  # Resting potential
V_thresh = -50e-3  # Threshold potential
V_reset = -80e-3  # Reset potential
R = 1e7  # Membrane resistance
dt = 1e-4  # Time step
T = 1.0  # 1 second simulation

num_neurons_input = 240  # Number of input neurons (same as sensor width)
num_neurons_hidden = 100  # Number of hidden neurons
num_neurons_output = 3  # Number of output neurons (classes)

# Create synthetic event data for training
num_samples = 100
data = [normalize_events(generate_synthetic_events(num_events, sensor_width, sensor_height), sensor_width, sensor_height) for _ in range(num_samples)]
labels = np.random.randint(0, num_neurons_output, num_samples)

# Convert event data to input spikes
input_spikes_data = [events_to_input(events, num_neurons_input, T, dt) for events in data]


## Step 3: Train

In [3]:
def train(network, data, labels, num_epochs, learning_rate):
    for epoch in range(num_epochs):
        for input_spikes, label in zip(data, labels):
            network.reset()
            for t in range(input_spikes.shape[1]):
                network.forward(input_spikes[:, t])

            target_spikes = np.zeros(network.output_layer.num_neurons)
            target_spikes[label] = 1

            dW_hidden_output = learning_rate * (target_spikes - network.output_layer.spikes).reshape(-1, 1) @ network.hidden_layer.spikes.reshape(1, -1)
            network.W_hidden_output += dW_hidden_output

            dW_input_hidden = learning_rate * (network.hidden_layer.spikes.reshape(-1, 1) @ input_spikes[:, t].reshape(1, -1))
            network.W_input_hidden += dW_input_hidden

        print(f'Epoch {epoch + 1}/{num_epochs} completed.')

# Create and train the network
network = SpikingNeuralNetwork(num_neurons_input, num_neurons_hidden, num_neurons_output, tau_m, V_rest, V_thresh, V_reset, R, dt)
train(network, input_spikes_data, labels, num_epochs=10, learning_rate=0.01)


Epoch 1/10 completed.
Epoch 2/10 completed.
Epoch 3/10 completed.
Epoch 4/10 completed.
Epoch 5/10 completed.
Epoch 6/10 completed.
Epoch 7/10 completed.
Epoch 8/10 completed.
Epoch 9/10 completed.
Epoch 10/10 completed.


## Step 4: Evaluate

In [4]:
def evaluate(network, data, labels):
    correct = 0
    total = len(labels)

    for input_spikes, label in zip(data, labels):
        network.reset()
        for t in range(input_spikes.shape[1]):
            network.forward(input_spikes[:, t])

        predicted_label = np.argmax(network.output_layer.spikes)
        if predicted_label == label:
            correct += 1

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')

# Create synthetic event data for testing
test_data = [normalize_events(generate_synthetic_events(num_events, sensor_width, sensor_height), sensor_width, sensor_height) for _ in range(num_samples)]
test_labels = np.random.randint(0, num_neurons_output, num_samples)

# Convert event data to input spikes for testing
test_input_spikes_data = [events_to_input(events, num_neurons_input, T, dt) for events in test_data]

# Evaluate the network
evaluate(network, test_input_spikes_data, test_labels)


Accuracy: 30.00%
